In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
import operator
import math
import hashlib

#load dataset from csv file into pandas dataframe
def load_data():
    return pd.read_csv("iris.csv")

iris_data=load_data()
iris_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
sepal_length    150 non-null float64
sepal_width     150 non-null float64
petal_length    150 non-null float64
petal_width     150 non-null float64
species         150 non-null object
dtypes: float64(4), object(1)
memory usage: 5.9+ KB


In [2]:
def test_set_check(identifier,test_ratio,hash):
    return bytearray(hash(np.int64(identifier)).digest())[-1]  < 51

def split_train_test_by_id(data,test_ratio,id_column,hash):
    ids=data[id_column]
    in_test_set=ids.apply(lambda id_:test_set_check(id_,test_ratio,hash))
    return in_test_set

In [3]:
iris_data=iris_data.reset_index()
in_test_set=split_train_test_by_id(iris_data,0.66,"index",hashlib.md5)
test_set=iris_data[in_test_set]
train_set=iris_data[~in_test_set]
del train_set['index']
del test_set['index']

In [4]:
train_set_array=train_set.as_matrix()
test_set_copy=test_set.copy()
del test_set['species']
test_set_array=test_set.as_matrix()

In [5]:
def euclideanDistance(instance1, instance2, length):
    distance = 0
    for x in range(length):
        distance += pow((instance1[x] - instance2[x]), 2)
    return math.sqrt(distance)

In [6]:
def getNeighbors(trainingSet, testInstance, k):
    distances = []
    length = len(testInstance)
    for x in range(len(trainingSet)):
        dist = euclideanDistance(testInstance, trainingSet[x], length)
        distances.append((trainingSet[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

In [7]:
def getResponse(neighbors):
    classVotes = {}
    for x in range(len(neighbors)):
        response = neighbors[x][4]
        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 1
    sortedVotes = sorted(classVotes.iteritems(), key=operator.itemgetter(1), reverse=True)
    print(sortedVotes)
    return sortedVotes[0][0]


In [8]:
k=1
pred=[]
for x in range(len(test_set_array)):
    neighbours=getNeighbors(train_set_array,test_set_array[x],k)
    pred.append(neighbours[0][4])

In [9]:
def getAccuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] is predictions[x]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0

In [10]:
test_set_copy=test_set_copy.as_matrix()
accuracy=getAccuracy(test_set_copy,pred)
print(accuracy)

92.3076923076923
